# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

Recall Problem 1 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [269]:
import networkx as nx

G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [270]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [271]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [272]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    eq_graph = graph.copy()
    eq_graph.add_node('s') #create a new source node
    eq_graph.add_node('t') #create a new sink node
    
    for node in graph.nodes(): #set edges capacity for s and t
        if graph.nodes[node]['demand'] < 0:
            eq_graph.add_edge('s',node)
            eq_graph['s'][node]['capacity'] = -graph.nodes[node]['demand']
        if graph.nodes[node]['demand'] > 0:
            eq_graph.add_edge(node,'t')
            eq_graph[node]['t']['capacity'] = graph.nodes[node]['demand']
    
    max_flow, flow_dict = nx.maximum_flow(eq_graph,'s','t') #get the max flow and the flow dict of dicts
    flow_dict.pop('s')
    flow_dict.pop('t')

    demand = 0
    for node in graph.nodes(): # calculate the total demand of the graph
        if graph.nodes[node]['demand'] > 0:
            demand += graph.nodes[node]['demand']
    
    for i in flow_dict: #remove the s and t nodes from the dict of dicts
        if 's' in flow_dict[i]:
            flow_dict[i].pop('s')
        if 't' in flow_dict[i]:
            flow_dict[i].pop('t')

            
    if max_flow == demand: #check if we can find the max flow or not
        return flow_dict
    else: raise nx.NetworkXUnfeasible('NetworkXUnfeasible: There is no flow satisfying the demands.')

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [275]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function 
    out = {k: 0 for k in flow.keys()}
    into = {k: 0 for k in flow.keys()}
    
    for x in flow.keys():
        out[x] = sum(flow[x].values())
        for y in flow[x].keys():
            into[y] += flow[x][y]
            
    net = {k: 0 for k in flow.keys()}
    for k in flow.keys():
        net[k] = into[k] - out[k]
    return net

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [276]:
flow = flow_with_demands(G)
div = divergence(flow)

Flow satisfies all demands: True
